In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.master("local[*]").appName("groupby").getOrCreate()

In [38]:
flight_schema = StructType([StructField("airline", StringType(), True), StructField("flight", StringType(), True), \
                            StructField("source_city", StringType(), True), StructField("departure_time", StringType(), True), \
                            StructField("stops", StringType(), True), StructField("arrival_time", StringType(), True), \
                            StructField("destination_city", StringType(), True), StructField("class", StringType(), True), \
                            StructField("duration", IntegerType(), True), StructField("days_left", IntegerType(), True), StructField("price", IntegerType(), True)])

df = spark.read.format("csv").option("header", "true").option("inferSchema", "False").schema(flight_schema).load("/home/manish/Documents/VSCodeProjects/Spark-Functions/SampleData/flight_data.csv")
df.show(5)
df.dtypes

+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    null|        1| 5953|
|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    null|        1| 5953|
| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    null|        1| 5956|
| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    null|        1| 5955|
| Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    null|        1| 5955|
+--------+-------+-----------+--------------+-----+-------------+-------

[('airline', 'string'),
 ('flight', 'string'),
 ('source_city', 'string'),
 ('departure_time', 'string'),
 ('stops', 'string'),
 ('arrival_time', 'string'),
 ('destination_city', 'string'),
 ('class', 'string'),
 ('duration', 'int'),
 ('days_left', 'int'),
 ('price', 'int')]

In [55]:
from pyspark.sql import functions as Fun

df_1 = df.groupBy("airline").agg(Fun.sum("price").alias("toal_price"))
df_1.show()

+---------+----------+
|  airline|toal_price|
+---------+----------+
|   Indigo| 229580207|
| SpiceJet|  55681482|
|Air_India|1901529790|
|  AirAsia|  65858089|
| GO_FIRST| 130973972|
|  Vistara|3886552320|
+---------+----------+



In [56]:
from pyspark.sql.window import Window
from pyspark.sql import functions as Fun

window_fun = Window.partitionBy("airline").orderBy("source_city")
df_2 = df.select(df.airline, Fun.sum("price").over(window_fun).alias("total_price"))
df_2.show(5)

+-------+-----------+
|airline|total_price|
+-------+-----------+
|AirAsia|   14205456|
|AirAsia|   14205456|
|AirAsia|   14205456|
|AirAsia|   14205456|
|AirAsia|   14205456|
+-------+-----------+
only showing top 5 rows

